In [2]:
import {InferenceSession, Tensor} from 'onnxruntime-web';
const ndarray = require('ndarray')
const ops = require('ndarray-ops')
const fs = require('fs')
var text = fs.readFileSync("classes.txt").toString('utf-8');
const classes = text.split("\r\n");


SyntaxError: invalid syntax (2209280100.py, line 1)

In [ ]:

var path = 'https://farm2.staticflickr.com/1533/26541536141_41abe98db3_z_d.jpg'
var imageData = null;

: 

In [3]:
var imageData = await jimp.read(path).then(image => {
    return image.resize(224, 224) // resize
    //console.log(imageData.bitmap)
      //.quality(60) // set JPEG quality
      //.greyscale() // set greyscale
      //.write('./data/bird-small-bw.jpg'); // save
  })
  .catch(err => {
    console.error(err);
  });

In [4]:
function imageDataToTensor(data, dims): any {
    // 1a. Extract the R, G, and B channels from the data to form a 3D int array
    const [R, G, B] = new Array([], [], []);
    for (let i = 0; i < data.length; i += 4) {
      R.push(data[i]);
      G.push(data[i + 1]);
      B.push(data[i + 2]);
      // 2. skip data[i + 3] thus filtering out the alpha channel
    }
    ///console.log(R);
    //console.log(G);
    //console.log(B);
    // 1b. concatenate RGB ~= transpose [224, 224, 3] -> [3, 224, 224]
    const transposedData = R.concat(G).concat(B);

    // 3. convert to float32
    let i, l = transposedData.length; // length, we need this for the loop
    const float32Data = new Float32Array(3 * 224 * 224); // create the Float32Array for output
    for (i = 0; i < l; i++) {
      float32Data[i] = transposedData[i] / 255.0; // convert to float
    }
  
    const inputTensor = new Tensor("float32", float32Data, dims);
    return inputTensor;
  }

In [5]:
var data = imageDataToTensor(imageData.bitmap.data, [1, 3, 224, 224])
// create an inference session, using WebGL backend. (default is 'wasm') 
//const session = await ort.InferenceSession.create('./model/squeezenet1_1.onnx', { executionProviders: ['wasm'] }); 
const session = await InferenceSession.create('../model/resnet50v2.onnx', { executionProviders: ['wasm'] });


In [6]:

async function runModel(model, preprocessedData): Promise<[Tensor, number]> {
    const start = new Date();
    try {
      const feeds: Record<string, Tensor> = {};
      feeds[model.inputNames[0]] = preprocessedData;
      const outputData = await model.run(feeds);
      const end = new Date();
      const inferenceTime = (end.getTime() - start.getTime());
      const output = outputData[model.outputNames[0]];
      return [output, inferenceTime];
    } catch (e) {
      console.error(e);
      throw new Error();
    }
  }

In [7]:
//The softmax transforms values to be between 0 and 1
function softmax(resultArray: number[]): any {
  // Get the largest value in the array.
  const largestNumber = Math.max(...resultArray);
  // Apply exponential function to each result item subtracted by the largest number, use reduce to get the previous result number and the current number to sum all the exponentials results.
  const sumOfExp = resultArray.map((resultItem) => Math.exp(resultItem - largestNumber)).reduce((prevNumber, currentNumber) => prevNumber + currentNumber);
  //Normalizes the resultArray by dividing by the sum of all exponentials; this normalization ensures that the sum of the components of the output vector is 1.
  return resultArray.map((resultValue, index) => {
    return Math.exp(resultValue - largestNumber) / sumOfExp;
  });
}

In [8]:
const [res, time] =  await runModel(session, data);
var output = res.data;
var inferenceTime = time;
var results = softmax(Array.prototype.slice.call(output))

In [4]:
let startingArray = [true, false, false,
  true, true, true,
  true, true, true,
  true, false, false,
  false, false, false
];
let cols = 5;
let rows = 3;

console.log(startingArray);
console.log(arrayToImageData(startingArray, cols, rows));

function arrayToImageData(input: any, width: number, height: number) {
  // const [r, g, b, a] = [0, 114, 189, 255]; // the masks's blue color
  let arr: number[] = [];
  for (let i = 0; i < input.length; i++) {

    // Threshold the onnx model mask prediction at 0.0
    // This is equivalent to thresholding the mask using predictor.model.mask_threshold
    // in python
    if (input[i] <= 0.0) {
      input[i] = 0;
    }
    else {
      input[i] = 1;
    }
  }
  console.log(
    `before transposed array: ${input.reduce(
      (partialSum: number, a: number) => partialSum + a,
      0,
    )}`,
  );
  for (let i = 0; i < height; i++) {
    for (let j = width-1; j >= 0; j--) {
      // let idxWidth = i * width + j;
      // let idxHeight = j * height + i;
      arr.push(input[i * width + j]);
    }
  }

  let transposedInput = new Float32Array(arr);

  console.log(
    `sum of transposed array: ${transposedInput.reduce(
      (partialSum, a) => partialSum + a,
      0,
    )}`,
  );
  return transposedInput;
}

// 0 1 1 0 0
// 0 1 1 0 0
// 1 1 1 1 0

[
  true,  false, false,
  true,  true,  true,
  true,  true,  true,
  true,  false, false,
  false, false, false
]
before transposed array: 8
sum of transposed array: 8
Float32Array(15) [
  1, 1, 0, 0, 1, 1,
  1, 1, 1, 1, 0, 0,
  0, 0, 0
]


In [15]:
let startingArray = [0, 0, 0, 0, 0, 0,
  0, 0, 1, 1, 1, 0,
  0, 1, 1, 1, 1, 0,
  0, 1, 1, 1, 1, 0,
  0, 0, 1, 1, 0, 0,
  0, 0, 1, 0, 0, 0,
  0, 0, 0, 0, 0, 0
];
let cols = 6;
let rows = 7;

console.log(startingArray);
console.log(duplicate(startingArray));
console.log(flipVertical(startingArray));
console.log(flipHorizontal(startingArray));
console.log(rotate90Clockwise(startingArray));
console.log(rotate(startingArray));
console.log(rotateColMajor(startingArray));


function duplicate(arr) {
  var result: any = [];
  for (var r = 0; r < rows; r++) {
    for (var c = 0; c < cols; c++) {
      result.push(arr[c * rows + r]);
    }
  }
  return result;
}

function flipVertical(arr) {
  var result: any = [];
  for (var r = rows - 1; r >= 0; r--) {
    for (var c = 0; c < cols; c++) {
      result.push(arr[c * rows + r]);
    }
  }
  return result;
}

function flipHorizontal(arr) {
  var result: any = [];
  for (var r = 0; r < rows; r++) {
    for (var c = cols - 1; c >= 0; c--) {
      result.push(arr[r * cols + c]);
    }
  }
  return result;
}

function rotate90Clockwise(arr) {
  var result: any = [];
  for (var r = 0; r < rows; r++) {
    for (var c = cols - 1; c >= 0; c--) {
      result.push(arr[c * rows + r]);
    }
  }
  return result;
}

function rotate90Counterclockwise(arr) {
  var result: any = [];
  for (var r = 0; r < rows; r++) {
    for (var c = cols - 1; c >= 0; c--) {
      result.push(arr[c * rows + r]);
    }
  }
  return result;
}

function rotate(arr) {
  var result: any = [];
  for (let c = 0; c < cols; c++) {
    for (let r = rows - 1; r >= 0; r--) {
      result.push(arr[r * cols + c]);
    }
  }
  return result;
}

function rotateColMajor(arr) {
  let result: any = [];
  for (let c = cols - 1; c >= 0; c--) {
    for (let r = 0; r < rows; r++) {
      result.push(arr[r * cols + c]);
    }
  }
  return result;
}
// 0 1 1 0 0
// 0 1 1 0 0
// 1 1 1 1 0

[
  0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
  1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
  1, 1, 1, 0, 0, 0, 1, 1, 0, 0,
  0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
  0, 0
]
[
  0, 0, 1, 1, 0, 0, 0, 1, 1, 1,
  0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
  0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
  0, 0, 1, 1, 0, 0, 0, 1, 1, 1,
  0, 0
]
[
  0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
  0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
  0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
  0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
  0, 0
]
[
  0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
  0, 0, 0, 1, 1, 1, 1, 0, 0, 1,
  1, 1, 1, 0, 0, 0, 1, 1, 0, 0,
  0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
  0, 0
]
[
  0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
  1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
  0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
  0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
  1, 0
]
[
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
  0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
  0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
  0, 0
]
[
  0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
  1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
  0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
  1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0
]


In [20]:
function rotateCounterClockwise(arr, width, height) {
    const rotatedArray: any = [];
  
    for (let col = width - 1; col >= 0; col--) {
      for (let row = 0; row < height; row++) {
        rotatedArray.push(arr[row * width + col]);
      }
    }
  
    return rotatedArray;
  }
  
  function flattenImage(arr, width, height) {
    const flattenedArray: any = [];
    for (let row = 0; row < height; row++) {
      for (let col = 0; col < width; col++) {
        flattenedArray.push(arr[row * width + col]);
      }
    }
    return flattenedArray;
  }
  
  // Given 2D image represented as column-major array
  const columnMajorArray = [
    0, 0, 0, 0, 0, 0,
    0, 0, 1, 1, 1, 0,
    0, 1, 1, 1, 1, 0,
    0, 1, 1, 1, 1, 0,
    0, 0, 1, 1, 0, 0,
    0, 0, 1, 0, 0, 0,
    0, 0, 0, 0, 0, 0,
  ];
  
  const width = 6;
  const height = 7;
  
  // Rotate 90 degrees counter-clockwise
  const rotatedArray = rotateCounterClockwise(columnMajorArray, width, height);
  
  // Flatten the rotated 2D image back to column-major array
  const flattenedArray = flattenImage(rotatedArray, height, width);

  const rotate = rotateCounterClockwise(flattenedArray, height, width)
  
  console.log(flattenedArray);

[
  0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
  1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
  0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
  1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0
]


In [21]:
function rotateCounterClockwise(arr, width, height) {
    const rotatedArray: any = [];
  
    for (let col = 0; col < width; col++) {
      for (let row = height - 1; row >= 0; row--) {
        rotatedArray.push(arr[row * width + col]);
      }
    }
  
    return rotatedArray;
  }
  
  function flattenImage(arr, width, height) {
    const flattenedArray: any = [];
    for (let col = 0; col < width; col++) {
      for (let row = 0; row < height; row++) {
        flattenedArray.push(arr[row * width + col]);
      }
    }
    return flattenedArray;
  }
  
  // Given 2D image represented as column-major array
  const columnMajorArray = [
    0, 0, 0, 0, 0, 0,
    0, 0, 1, 1, 1, 0,
    0, 1, 1, 1, 1, 0,
    0, 1, 1, 1, 1, 0,
    0, 0, 1, 1, 0, 0,
    0, 0, 1, 0, 0, 0,
    0, 0, 0, 0, 0, 0,
  ];
  
  const width = 6;
  const height = 7;
  
  // Rotate 90 degrees counter-clockwise
  const rotatedArray = rotateCounterClockwise(columnMajorArray, width, height);
  
  // Flatten the rotated 2D image back to column-major array
  const flattenedArray = flattenImage(rotatedArray, height, width);
  
  console.log(flattenedArray);
  

[
  0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
  0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
  1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
  0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
  0, 0
]


In [ ]:
function createColumnMajorArray(mask, width, height) {
    const columnMajorArray: any = [];
  
    for (let col = 0; col < width; col++) {
      for (let row = 0; row < height; row++) {
        columnMajorArray.push(mask[row * width + col]);
      }
    }
  
    return columnMajorArray;
  }
  
  // Given binary mask image
  const binaryMaskImage = [
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  ];
  
  const width = 24;
  const height = 4;
  
  // Create the 1D column-major array from the binary mask image
  const columnMajorArray = createColumnMajorArray(binaryMaskImage, width, height);
  
  console.log(columnMajorArray);

: 